## Testing PyNbody

In [1]:
# basic packages
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
#import databricks.koalas as ks

# the default `sequence` option merges all partitions into a single catastrophic one : what?
#ks.set_option('compute.default_index_type', 'distributed') 

# plot settings
#plt.rc('text', usetex=True )
#plt.rc('font', family='Times New Roman', weight='normal', size=14)
#plt.rcParams['mathtext.fontset'] = 'stix'

plt.rc('font', family='serif') 
plt.rc('font', serif='Times New Roman') 
plt.rcParams['mathtext.fontset'] = 'cm'  
plt.rcParams.update({'font.size': 16})

In [2]:
!pwd

/home/shong/work/panspermia/notebook


In [3]:
!ls -alF 

total 32
drwxrwxr-x 3 shong shong  4096 10월 18 15:12 ./
drwxrwxr-x 5 shong shong  4096 10월 17 15:21 ../
drwxrwxr-x 2 shong shong  4096 10월 17 15:44 .ipynb_checkpoints/
-rw-rw-r-- 1 shong shong 16489 10월 18 15:12 pynbody-read-data.ipynb


### Read the snapshot file

In [4]:
import pynbody

In [5]:
!ls -alF /home/shong/work/panspermia/data/

total 7500724
drwxrwxr-x 2 shong shong       4096 10월 18 15:11 ./
drwxrwxr-x 5 shong shong       4096 10월 17 15:21 ../
-rw-r--r-- 1 shong shong      16384 10월 18 11:32 .h277.cosmo50cmb.3072g14HMbwK.00512.param.swp
-rw-rw-r-- 1 shong shong   16711680  9월 29 15:33 h277.3072g14HMbwK.3.mpg
-rw-rw-r-- 1 shong shong 1411600636  9월 29 15:39 h277.cosmo50cmb.3072g14HMbwK.00512
-rw-rw-r-- 1 shong shong      30280  9월 29 15:37 h277.cosmo50cmb.3072g14HMbwK.00512.00.log
-rw-rw-r-- 1 shong shong      29652 10월 17 15:50 h277.cosmo50cmb.3072g14HMbwK.00512.0000.z0.000.AHF_fpos
-rw-rw-r-- 1 shong shong    2877901  9월 29 15:30 h277.cosmo50cmb.3072g14HMbwK.00512.0000.z0.000.AHF_halos
-rw-rw-r-- 1 shong shong  207516366  9월 29 15:32 h277.cosmo50cmb.3072g14HMbwK.00512.0000.z0.000.AHF_particles
-rw-rw-r-- 1 shong shong    9634825  9월 29 15:32 h277.cosmo50cmb.3072g14HMbwK.00512.0000.z0.000.AHF_profiles
-rw-rw-r-- 1 shong shong   89017954  9월 29 15:32 h277.cosmo50cmb.3072g14HMbwK.00512.ESNRate
-rw-rw-r-- 1 sho

In [6]:
s = pynbody.load('/home/shong/work/panspermia/data/h277.cosmo50cmb.3072g14HMbwK.00512',\
                paramname='/home/shong/work/panspermia/data/h277.cosmo50cmb.3072g14HMbwK.00512.param')

/home/shong/anaconda3/lib/python3.7/site-packages/pynbody/snapshot/tipsy.py:125: RuntimeWarning: No readable param file in the run directory or parent directory: using defaults.
  "No readable param file in the run directory or parent directory: using defaults.", RuntimeWarning)


In [7]:
sys.getsizeof(s)

64

In [8]:
s.keys()

[]

In [9]:
s.loadable_keys()

['pos',
 'lw',
 'ESNRate',
 'HeI',
 'iord',
 'eps',
 'HeII',
 'phi',
 'H2',
 'mass',
 'vel',
 'igasorder',
 'HI',
 'FeMassFrac',
 'coolontime',
 'OxMassFrac']

In [10]:
s.properties

{'time': Unit("1.00e-05 kpc**3/2 G**-1/2 Msol**-1/2")}

In [11]:
s.families()

[<Family gas>, <Family dm>, <Family star>]

In [12]:
s.star.loadable_keys()

['iord',
 'tform',
 'eps',
 'H2',
 'FeMassFrac',
 'coolontime',
 'HeI',
 'HeII',
 'pos',
 'lw',
 'ESNRate',
 'metals',
 'phi',
 'mass',
 'vel',
 'igasorder',
 'HI',
 'OxMassFrac']

In [13]:
len(s.star)

8115217

In [14]:
s.star['pos'][:3]

SimArray([[0.09565262, 0.09649316, 0.02786732],
          [0.10488313, 0.10036243, 0.02527858],
          [0.1144229 , 0.11139609, 0.02712537]], 'kpc')

In [15]:
s.physical_units()

In [16]:
s.star['pos'][:3]

SimArray([[0.09565262, 0.09649316, 0.02786732],
          [0.10488313, 0.10036243, 0.02527858],
          [0.1144229 , 0.11139609, 0.02712537]], 'kpc')

In [17]:
s.star['mass'][:10]

SimArray([0.00307807, 0.00307807, 0.00307807, 0.00307807, 0.00307807,
          0.00307807, 0.00307807, 0.00307807, 0.00307807, 0.00307807], 'Msol')

In [18]:
s.star['pos'][0][0]

0.09565262496471405

In [19]:
s.star['pos'][0:6].view(type=np.ndarray)[1]

array([0.10488313, 0.10036243, 0.02527858])

In [20]:
s.star['pos'].view(type=np.ndarray)[0:6].transpose()[0]

array([0.09565262, 0.10488313, 0.1144229 , 0.07517252, 0.09367973,
       0.07872621])

In [21]:
starpos = s.star['pos'].view(type=np.ndarray).transpose()

In [22]:
len(starpos)

3

In [23]:
len(starpos[0])

8115217

### Pandas DataFrame

In [24]:
stardf = pd.DataFrame(s.star['pos'].view(type=np.ndarray), columns = ['px','py','pz'])

In [25]:
stardf.head(3)

,px,py,pz
0,0.095653,0.096493,0.027867
1,0.104883,0.100362,0.025279
2,0.114423,0.111396,0.027125


### Save it to a Parquet

In [26]:
import pyarrow as pa
import pyarrow.parquet as pq

In [27]:
!pwd

/home/shong/work/panspermia/notebook


In [29]:
pq.write_table(pa.Table.from_pandas(stardf), \
               '/home/shong/work/panspermia/data/starpos.parquet.snappy', compression='snappy')

In [30]:
!ls ../data

h277.3072g14HMbwK.3.mpg
h277.cosmo50cmb.3072g14HMbwK.00512
h277.cosmo50cmb.3072g14HMbwK.00512.00.log
h277.cosmo50cmb.3072g14HMbwK.00512.0000.z0.000.AHF_fpos
h277.cosmo50cmb.3072g14HMbwK.00512.0000.z0.000.AHF_halos
h277.cosmo50cmb.3072g14HMbwK.00512.0000.z0.000.AHF_particles
h277.cosmo50cmb.3072g14HMbwK.00512.0000.z0.000.AHF_profiles
h277.cosmo50cmb.3072g14HMbwK.00512.ESNRate
h277.cosmo50cmb.3072g14HMbwK.00512.FeMassFrac
h277.cosmo50cmb.3072g14HMbwK.00512.H2
h277.cosmo50cmb.3072g14HMbwK.00512.HI
h277.cosmo50cmb.3072g14HMbwK.00512.HeI
h277.cosmo50cmb.3072g14HMbwK.00512.HeII
h277.cosmo50cmb.3072g14HMbwK.00512.OxMassFrac
h277.cosmo50cmb.3072g14HMbwK.00512.coolontime
h277.cosmo50cmb.3072g14HMbwK.00512.igasorder
h277.cosmo50cmb.3072g14HMbwK.00512.iord
h277.cosmo50cmb.3072g14HMbwK.00512.lw
h277.cosmo50cmb.3072g14HMbwK.00512.param
h277.cosmo50cmb.3072g14HMbwK.00512.parameter
h277.cosmo50cmb.3072g14HMbwK.log
h277.g14.00512.tar.gz
starpos.parquet.snappy
